# Deep Learning Assignment — Part 1: Custom CNNs
 
| | |
|---|---|
| **Students** | Gianluca Lascaro, Raffaele Rizzuti |
| **University** | Universidad de A Coruña |
| **Academic Year** | 2025–2026 |
 


# 1. Imports & Environment Setup

In [5]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
 
# Reproducibility
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

## 2. Hyperparameters & Configuration

Centralising all configuration constants here makes it easy to tune them later.

In [6]:
# ── Image settings ──────────────────────────────────────────────────────────
TARGET_SIZE  = (96, 96)    # STL-10 native resolution; change if you resize
NUM_CHANNELS = 3           # RGB
NUM_CLASSES  = 10

# ── Dataset split ───────────────────────────────────────────────────────────
VALIDATION_SPLIT = 0.2     # 20% of training set → validation

# ── Training settings (used later) ──────────────────────────────────────────
BATCH_SIZE = 32
AUTOTUNE   = tf.data.AUTOTUNE

### 1.1 Loading the STL-10 Dataset

The **STL-10** dataset is loaded directly via TensorFlow Datasets. It includes:

- 5,000 labeled training images (500 per class)  
- 8,000 test images  
- 100,000 unlabeled images (not used here)  

Each image has size **96 × 96 × 3 (RGB)**.

The code performs the following steps:

- Loads the full dataset with metadata (`info`).  
- Splits the labeled training set into:
  - **Training:** 80% (4,000 images)  
  - **Validation:** 20% (1,000 images)  
- Loads the full test set (8,000 images).


In [ ]:
# Load the dataset
dataset, info = tfds.load('stl10', with_info=True, as_supervised=True)

# Split the dataset into training,validation and test sets
train_split = tfds.load('stl10', split='train[:80%]', as_supervised=True)
val_split = tfds.load('stl10', split='train[80%:]', as_supervised=True)
test_split = tfds.load('stl10', split='test', as_supervised=True)

### 1.2 Exploratory Data Analysis (EDA)

A quick sanity-check before any preprocessing.

In [9]:
# Dataset sizes
num_train = ds_info.splits['train'].num_examples
num_test  = ds_info.splits['test'].num_examples
print(f"Training samples : {num_train}")
print(f"Test samples     : {num_test}")

# Class names
class_names = ds_info.features['label'].names
print(f"Classes ({NUM_CLASSES}): {class_names}")

NameError: name 'ds_info' is not defined

In [ ]:
# Visualise a few raw samples
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for ax, (image, label) in zip(axes.flat, ds_train_full.take(10)):
    ax.imshow(image.numpy())
    ax.set_title(class_names[label.numpy()], fontsize=9)
    ax.axis('off')
plt.suptitle('Sample raw images from STL-10 training set', fontsize=12)
plt.tight_layout()
plt.show()

### 1.3 Preprocessing Pipeline

We define a single `preprocess` function that will be mapped over every split.  
Steps applied:

1. **Resize** — standardise all images to `TARGET_SIZE`
2. **Normalize** — scale pixel values from `[0, 255]` → `[0.0, 1.0]`
3. **One-hot encode** — convert integer labels to categorical vectors of length `NUM_CLASSES`

In [ ]:
def preprocess(image, label):
    """Resize, normalize and one-hot encode a single (image, label) pair."""
    # 1. Standardise image size
    image = tf.image.resize(image, TARGET_SIZE)

    # 2. Normalize to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0

    # 3. One-hot encode the label
    label = tf.one_hot(label, depth=NUM_CLASSES)

    return image, label

## 6. Train / Validation Split

The STL-10 TFDS API does not provide a built-in validation split, so we carve 
one out of the training set manually using `take` and `skip`.

Although STL-10 is perfectly balanced by construction (500 images per class),
we shuffle **before** splitting to avoid any class-ordering artifacts that may
arise from the way TFDS loads the dataset on disk, ensuring the validation set
is a representative sample of the full training distribution.

In [ ]:
# Compute split sizes
num_val          = int(num_train * VALIDATION_SPLIT)
num_train_split  = num_train - num_val

print(f"Train subset size      : {num_train_split}")
print(f"Validation subset size : {num_val}")

# Shuffle BEFORE splitting to ensure class balance
ds_train_full = ds_train_full.shuffle(
    buffer_size=num_train, seed=SEED, reshuffle_each_iteration=False
)

ds_val   = ds_train_full.take(num_val)
ds_train = ds_train_full.skip(num_val)

## 7. Build Final tf.data Pipelines

We apply preprocessing and configure efficient pipelines with batching and prefetching.

In [ ]:
ds_train = (
    ds_train
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

ds_val = (
    ds_val
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

ds_test = (
    ds_test
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

print("Datasets ready:")
print(f"  ds_train : {ds_train}")
print(f"  ds_val   : {ds_val}")
print(f"  ds_test  : {ds_test}")

## 8. Sanity Check — Preprocessed Samples

Verify shapes, value ranges, and label format after preprocessing.

In [ ]:
for images, labels in ds_train.take(1):
    print(f"Image batch shape : {images.shape}")
    print(f"Label batch shape : {labels.shape}")
    print(f"Pixel min / max   : {images.numpy().min():.4f} / {images.numpy().max():.4f}")
    print(f"Example label (one-hot): {labels[0].numpy()}")

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for images, labels in ds_train.take(1):
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].numpy())
        ax.set_title(class_names[np.argmax(labels[i].numpy())], fontsize=9)
        ax.axis('off')
plt.suptitle('Preprocessed training samples', fontsize=12)
plt.tight_layout()
plt.show()